<a href="https://colab.research.google.com/github/ukhyun-lee/stock_market_analysis/blob/main/AI%EC%A4%91%EA%B8%89/AI%EC%8B%A4%EC%8A%B5_8%EC%9B%9423%EC%9D%BC(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall torchtext

Found existing installation: torchtext 0.15.2
Uninstalling torchtext-0.15.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchtext-0.15.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchtext/*
Proceed (Y/n)? Y
  Successfully uninstalled torchtext-0.15.2


In [ ]:
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.2 MB/s eta 0:00:00


In [ ]:
import torch
import torchtext
from torchtext.data import Field

import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [ ]:
start = time.time()
TEXT = Field(lower=True, fix_length=200, batch_first=False)  #[seq_len, batch_size, hidden_size] True이면 [batch_size, seq_len, hidden_size]
LABEL = Field(sequential=False)

In [ ]:
from torchtext.datasets import IMDB
train_data, test_data = IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 72.7MB/s]


In [ ]:
vars(train_data.examples[0])

{'text': ['i',
  'saw',
  'this',
  'on',
  'tv',
  'so',
  'long',
  'ago',
  'that',
  'i',
  "can't",
  'remember',
  'when',
  'it',
  'was,',
  'but',
  'it',
  'still',
  'stands',
  'out',
  'as',
  'one',
  'of',
  'the',
  'scariest,',
  'most',
  'unnerving',
  'films',
  "i've",
  'ever',
  'seen.',
  'there',
  'is',
  'a',
  'simultaneously',
  'subtle',
  'but',
  'intense',
  'dread',
  'induced',
  'by',
  'the',
  'woman',
  'in',
  'black',
  'lurking',
  'at',
  'the',
  'edge',
  'of',
  'the',
  'frame,',
  'not',
  'quite',
  'clearly',
  'visible,',
  'so',
  'that',
  'you',
  'feel',
  '(like',
  'the',
  'solicitor',
  'hero),',
  'unsure',
  'whether',
  'its',
  'just',
  'imagination',
  'or',
  'not.',
  'it',
  'is',
  'also',
  'one',
  'of',
  'the',
  'few',
  'films',
  'which',
  'has',
  'really',
  'made',
  'me',
  'fearful',
  'to',
  'keep',
  'watching.',
  '"production',
  'values"',
  'be',
  'hanged,',
  'good',
  'films',
  'are',
  'about'

In [ ]:
import string

for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']]
    text = [x.replace("<br","") for x in text]
    text = [''.join(c for c in s if c not in string.punctuation) for s in text]
    text = [s for s in text if s]
    vars(example)['text'] = text

In [ ]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [ ]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)


In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 3


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7a8b27d120b0>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [ ]:
BATCH_SIZE = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embeding_dim = 100
hidden_size = 300


train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

In [ ]:
class RNNCell_Encoder(nn.Module):
    def __init__(self, input_dim, hidden_size):
        super(RNNCell_Encoder, self).__init__()
        self.rnn = nn.RNNCell(input_dim, hidden_size)

    def forward(self, inputs): #------ inputs는 입력 시퀀스로 (시퀀스 길이, 배치, 임베딩(seq,batch, embedding))의 형태를 갖습니다.
        bz = inputs.shape[1] #------ 배치를 가져옵니다.
        ht = torch.zeros((bz, hidden_size)).to(device)# ------ 배치와 은닉층 뉴런의 크기를 0으로 초기화
        for word in inputs:
            ht = self.rnn(word, ht) #------ ②
        return ht

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim) #------ ③
        self.rnn = RNNCell_Encoder(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss()   #다중분류
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

#모델 학습을 위한 함수 공식
##1.데이터로더에서 데이터를 가져와서 모델에 적용한 후 손실 함수를 적용하여 오차를 구함
##2.옵티마이저를 이용하여 파라미터(가중치, 바이어스 등)를 업데이트

In [ ]:
def training(epoch, model, trainloader, validloader):
    correct = 0
    total = 0
    running_loss = 0

    model.train()  #dropout 켜짐
    for b in trainloader:
        x, y = b.text, b.label
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            valid_correct += (y_pred == y).sum().item()
            valid_total += y.size(0)
            valid_running_loss += loss.item()

    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct / valid_total

    print('epoch: ', epoch,
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'valid_loss： ', round(epoch_valid_loss, 3),
          'valid_accuracy:', round(epoch_valid_acc, 3)
          )
    return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [ ]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch,model,train_iterator,valid_iterator)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)
end = time.time()
print(end-start)

epoch:  0 loss：  0.011 accuracy: 0.528 valid_loss：  0.011 valid_accuracy: 0.509
epoch:  1 loss：  0.011 accuracy: 0.534 valid_loss：  0.011 valid_accuracy: 0.51
epoch:  2 loss：  0.011 accuracy: 0.536 valid_loss：  0.011 valid_accuracy: 0.503
epoch:  3 loss：  0.011 accuracy: 0.545 valid_loss：  0.011 valid_accuracy: 0.511
epoch:  4 loss：  0.011 accuracy: 0.55 valid_loss：  0.011 valid_accuracy: 0.513
2676.221497774124


In [ ]:
epochs = 20
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch,model,train_iterator,valid_iterator)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)
end = time.time()
print(end-start)

epoch:  0 loss：  0.01 accuracy: 0.557 valid_loss：  0.011 valid_accuracy: 0.506
epoch:  1 loss：  0.01 accuracy: 0.567 valid_loss：  0.011 valid_accuracy: 0.505
epoch:  2 loss：  0.01 accuracy: 0.572 valid_loss：  0.011 valid_accuracy: 0.509
epoch:  3 loss：  0.01 accuracy: 0.581 valid_loss：  0.011 valid_accuracy: 0.533
epoch:  4 loss：  0.01 accuracy: 0.587 valid_loss：  0.011 valid_accuracy: 0.52
epoch:  5 loss：  0.01 accuracy: 0.59 valid_loss：  0.011 valid_accuracy: 0.548
epoch:  6 loss：  0.01 accuracy: 0.598 valid_loss：  0.012 valid_accuracy: 0.539
epoch:  7 loss：  0.01 accuracy: 0.61 valid_loss：  0.012 valid_accuracy: 0.518
epoch:  8 loss：  0.009 accuracy: 0.616 valid_loss：  0.012 valid_accuracy: 0.54
epoch:  9 loss：  0.01 accuracy: 0.609 valid_loss：  0.012 valid_accuracy: 0.518
epoch:  10 loss：  0.009 accuracy: 0.618 valid_loss：  0.012 valid_accuracy: 0.509
epoch:  11 loss：  0.009 accuracy: 0.622 valid_loss：  0.012 valid_accuracy: 0.531
epoch:  12 loss：  0.009 accuracy: 0.635 valid_loss：

In [ ]:
def evaluate(epoch, model, testloader):
    test_correct = 0
    test_total = 0
    test_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in testloader:
            x, y = b.text, b.label
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()

    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total

    print('epoch: ', epoch,
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
          )
    return epoch_test_loss, epoch_test_acc

In [ ]:
epochs = 5
test_loss = []
test_acc = []

for epoch in range(epochs):
    epoch_test_loss, epoch_test_acc = evaluate(epoch,
                                               model,
                                               test_iterator)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

end = time.time()
print(end-start)

epoch:  0 test_loss：  0.014 test_accuracy: 0.528
epoch:  1 test_loss：  0.014 test_accuracy: 0.528
epoch:  2 test_loss：  0.014 test_accuracy: 0.528
epoch:  3 test_loss：  0.014 test_accuracy: 0.528
epoch:  4 test_loss：  0.014 test_accuracy: 0.528
4220.269709825516


In [ ]:
epochs = 20
test_loss = []
test_acc = []

for epoch in range(epochs):
    epoch_test_loss, epoch_test_acc = evaluate(epoch,
                                               model,
                                               test_iterator)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

end = time.time()
print(end-start)

epoch:  0 test_loss：  0.014 test_accuracy: 0.528
epoch:  1 test_loss：  0.014 test_accuracy: 0.528
epoch:  2 test_loss：  0.014 test_accuracy: 0.528
epoch:  3 test_loss：  0.014 test_accuracy: 0.528
epoch:  4 test_loss：  0.014 test_accuracy: 0.528
epoch:  5 test_loss：  0.014 test_accuracy: 0.528
epoch:  6 test_loss：  0.014 test_accuracy: 0.528
epoch:  7 test_loss：  0.014 test_accuracy: 0.528
epoch:  8 test_loss：  0.014 test_accuracy: 0.528
epoch:  9 test_loss：  0.014 test_accuracy: 0.528
epoch:  10 test_loss：  0.014 test_accuracy: 0.528
epoch:  11 test_loss：  0.014 test_accuracy: 0.528
epoch:  12 test_loss：  0.014 test_accuracy: 0.528
epoch:  13 test_loss：  0.014 test_accuracy: 0.528
epoch:  14 test_loss：  0.014 test_accuracy: 0.528
epoch:  15 test_loss：  0.014 test_accuracy: 0.528
epoch:  16 test_loss：  0.014 test_accuracy: 0.528
epoch:  17 test_loss：  0.014 test_accuracy: 0.528
epoch:  18 test_loss：  0.014 test_accuracy: 0.528
epoch:  19 test_loss：  0.014 test_accuracy: 0.528
4789.76269